<a href="https://colab.research.google.com/github/quagmaire/mistral-finetune-2025/blob/main/tutorials/mistral_finetune_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


In [1]:
!conda create --name myenv python=3.10

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    ld_impl_linux-64-2.44      |       ha97dd6f_2         730 KB  conda-forge
    libexpat-2.7.1             |       hecca717_0          73 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.1.0              |       h767d61c_5         805 KB  conda-forge
    li

In [2]:
%%bash
source activate myenv
python3 -<<'EOF'
import sys
print(sys.version)
EOF

3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:45:41) [GCC 13.3.0]


## Installation

Clone the `mistral-finetune` repo:


In [3]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 472 (delta 211), reused 159 (delta 159), pack-reused 223 (from 2)
Receiving objects: 100% (472/472), 243.32 KiB | 20.28 MiB/s, done.
Resolving deltas: 100% (251/251), done.


Install all required dependencies:

In [4]:
!conda run --live-stream -n myenv python -m pip install -r /content/mistral-finetune/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 32.2 MB/s  0:00:10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 124.2 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 119.6 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 83.4 MB/s  0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 173.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 209.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 29.6 MB/s  0:00:10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 156.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 169.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 156.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 111.9 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 149

## Model download

In [6]:
pip install huggingface_hub

In [7]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-v0.3

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

In [9]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [10]:
%cd /content/

/content


In [11]:
# make a new directory called data
!mkdir -p data

In [12]:
# navigate to this data directory
%cd /content/data

/content/data


In [13]:
#download datasets
!wget --header="Authorization: Bearer $(cat ~/.cache/huggingface/token)" https://huggingface.co/datasets/baggettersol/bagsy-dataset/resolve/main/bagsy-train.jsonl
!wget --header="Authorization: Bearer $(cat ~/.cache/huggingface/token)" https://huggingface.co/datasets/baggettersol/bagsy-dataset/resolve/main/bagsy-eval.jsonl

--2025-09-27 17:59:05--  https://huggingface.co/datasets/baggettersol/bagsy-dataset/resolve/main/bagsy-train.jsonl
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.97, 13.35.202.121, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24833 (24K) [text/plain]
Saving to: ‘bagsy-train.jsonl’

bagsy-train.jsonl   100%[===================>]  24.25K  --.-KB/s    in 0s      

2025-09-27 17:59:05 (303 MB/s) - ‘bagsy-train.jsonl’ saved [24833/24833]

--2025-09-27 17:59:05--  https://huggingface.co/datasets/baggettersol/bagsy-dataset/resolve/main/bagsy-eval.jsonl
Resolving huggingface.co (huggingface.co)... 13.35.202.40, 13.35.202.34, 13.35.202.121, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2648 (2.6K) [text/plain]
Saving to: ‘bagsy-eval.jsonl’

bagsy-eval.jsonl    100%[===================>]   2.59

In [14]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


## Start training

In [15]:
%%bash
source activate myenv
python3 -<<'EOF'
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
EOF

In [130]:
%%bash
source activate myenv
python3 -<<'EOF'
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/bagsy-train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/bagsy-eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 32

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 24
batch_size: 1
num_microbatches: 8
max_steps: 150
optim:
  lr: 3.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 25
no_eval: False
ckpt_freq: 50
num_ckpt_keep: 2

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)
EOF

In [118]:
#downgrade numpy
%%bash
source activate myenv

pip install numpy==1.26.4

In [128]:
#fix that mistral bs

!conda run --live-stream -n myenv python -m pip install mistral-common==1.3.1 --force-reinstall

  Using cached mistral_common-1.3.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached pydantic-2.6.1-py3-none-any.whl.metadata (83 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.27.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.5 

In [131]:
# validate data

!conda run --live-stream -n myenv python -m utils.validate_data --train_yaml example.yaml

0it [00:00, ?it/s]Validating /content/data/bagsy-train.jsonl ...

100% 151/151 [00:00<00:00, 7897.10it/s]
1it [00:00, 51.11it/s]
No errors! Data is correctly formatted!
Stats for /content/data/bagsy-train.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:05:00",
        "data_tokens": 4078,
        "train_tokens": 28800,
        "epochs": "7.06",
        "max_steps": 150,
        "data_tokens_per_dataset": {
            "/content/data/bagsy-train.jsonl": "4078.0"
        },
        "train_tokens_per_dataset": {
            "/content/data/bagsy-train.jsonl": "28800.0"
        },
        "epochs_per_dataset": {
            "/content/data/bagsy-train.jsonl": "7.1"
        }
    }
}
0it [00:00, ?it/s]Validating /content/data/bagsy-eval.jsonl ...

100% 16/16 [00:00<00:00, 7652.09it/s]
1it [00:00, 414.25it/s]
No errors! Data is correctly formatted!


In [46]:
!mkdir -p /usr/local/envs/myenv/etc/conda/activate.d
!echo 'export CUDA_VISIBLE_DEVICES=0' > /usr/local/envs/myenv/etc/conda/activate.d/env_vars.sh

In [132]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
!rm -r /content/test_ultra

rm: cannot remove '/content/test_ultra': No such file or directory


In [133]:
# start training
!conda run --live-stream -n myenv torchrun --nproc-per-node 1 -m train example.yaml

args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/content/data/bagsy-train.jsonl', eval_instruct_data='/content/data/bagsy-eval.jsonl', instruct=InstructArgs(shuffle=True, dynamic_chunk_fn_call=True)), model_id_or_path='/content/mistral_models', run_dir='/content/test_ultra', optim=OptimArgs(lr=0.0003, weight_decay=0.1, pct_start=0.05), seed=0, num_microbatches=8, seq_len=24, batch_size=1, max_norm=1.0, max_steps=150, log_freq=1, ckpt_freq=50, save_adapters=True, no_ckpt=False, num_ckpt_keep=2, eval_freq=25, no_eval=False, checkpoint=True, world_size=1, wandb=WandbArgs(project=None, offline=False, key=None, run_name=None), mlflow=MLFlowArgs(tracking_uri=None, experiment_name=None), lora=LoraArgs(enable=True, rank=32, dropout=0.0, scaling=2.0))
2025-09-27 19:17:30 (UTC) - 0:00:02 - distributed - INFO - torch.cuda.device_count: 1
2025-09-27 19:17:30 (UTC) - 0:00:02 - distributed - INFO - CUDA_VISIBLE_DEVICES: 0
2025-09-27 19:17:30 (UTC) - 0:00:02 - distributed - INFO 

# Merge Checkpoints

In [134]:
#optional if you are uploading file

#!unzip /content/test_ultra.zip -d /content/
#!mv /content/content/* /content/
#!rm -rf /content/content/

In [135]:
#make merged directory for saving merged model

!mkdir -p /content/merged

In [136]:
#merge checkpoints

!conda run --live-stream -n myenv python3 /content/mistral-finetune/utils/merge_lora.py --initial_model_ckpt /content/mistral_models/consolidated.safetensors --lora_ckpt /content/test_ultra/checkpoints/checkpoint_000150/consolidated/lora.safetensors --dump_ckpt /content/merged/consolidated.safetensors --scaling 2.0

Merging to torch.bfloat16 precision...
100% 224/224 [00:13<00:00, 16.60it/s]
100% 291/291 [00:00<00:00, 776426.50it/s]
Merged checkpoint saved to /content/merged/consolidated.safetensors


In [95]:
#copy over the important files to complete the output

!cp /content/mistral_models/params.json /content/merged/params.json

!cp /content/mistral_models/tokenizer.model.v3 /content/merged/tokenizer.model.v3

## Inference

In [138]:
!conda run --live-stream -n myenv python -m pip install mistral_inference

In [139]:
%%bash
source activate myenv
python3 -<<'EOF'

import json

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, AssistantMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/merged/tokenizer.model.v3")
model = Transformer.from_folder("/content/merged")

#CONFIG
bagsy_prompt = "You are 'Bagsy', a hilariously unfiltered and entertaining live streamer. You stream on a platform with loose TOS where anything goes, so you occasionally drop edgy adult jokes and hilarious one-liners."
user_msg = "bootymuncher5: fuck you Bagsy, I hate you motherfucker"
#####

completion_request = ChatCompletionRequest(messages=[
    #SystemMessage(content=bagsy_prompt),
    UserMessage(content=user_msg)
])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.2, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)
EOF

Hate me all you want. I'm a 1 of 1, bitch.


/usr/local/envs/myenv/lib/python3.10/site-packages/mistral_common/tokens/tokenizers/sentencepiece.py:175: FutureWarning: Using the tokenizer's special token policy `None` is deprecated. It will be removed in 1.10.0. Please pass a special token policy explicitly. Future default will be SpecialTokenPolicy.IGNORE.
  warnings.warn(
